# Errors and Exceptions

There are (at least) two distinguishable kinds of errors: **syntax errors and exceptions**.

## Syntax Errors

Syntax eroors, as known as **parsing errors**, are perhaps the most common kind of complaint you get while learning Python.

In [1]:
while True print('Hello, world.')

SyntaxError: invalid syntax (<ipython-input-1-8cc31e9cce30>, line 1)

From the example above, we can conclude that:
- File name and line number are printed, so you know where to look in case the input came from a script.
- There will be a little ‘arrow’ pointing at the earliest point in the line where the error was detected.
- Error types and simple explanations are printed, and the first thing is to read them, instead of searching solutions via network or others. 

## Excptions

Errors detected during execution are call _excptions_ and are not unconditionally fatal.

In [2]:
1/0

ZeroDivisionError: division by zero

In [3]:
4 + spam *3

NameError: name 'spam' is not defined

In [4]:
'2' + 2

TypeError: must be str, not int

From the example above, we can conclude that:
- The preceding part of the error message shows **the context where the exception happened**, in the form of a **stack traceback**.
- The last line of the error message indicates what happend.
    - The **type of excption** is printed as the part of the message.
    - The rest of line provides **details based on the type of exception and what caused it**.

## Handling Excptions 

It is possible to write programs handle selected exceptions by using `try ... except` statement.

The try statement works as follows.
- First, the try clause (the statement(s) between the try and except keywords) is executed.
- If no exception occurs, the except clause is skipped and execution of the try statement is finished.
- If an exception occurs during execution of the try clause, the rest of the clause is skipped. Then if its type matches the exception named after the except keyword, the except clause is executed, and then execution continues after the try statement.
- If an exception occurs which does not match the exception named in the except clause, it is passed on to outer try statements; if no handler is found, it is an unhandled exception and execution stops with a message as shown above.

In [5]:
while True:
    try:
        x = int(input("please enter a number: "))
        break
    except ValueError:
        print("Oops! That was no valid number. Try again ...")

please enter a number: p
Oops! That was no valid number. Try again ...
please enter a number: 6


### Note :
- A try statement may have **more than one except clause**, to specify handlers for different exceptions.

In [6]:
try:
    pass
except (RuntimeError, TypeError, NameError):
    pass

- The last except clause may omit the exception name(s), to serve as a wildcard.

In [7]:
import sys
with open('myfile.txt', 'w') as f:
    f.write('this is my file')
try:
    f = open('myfile.txt')
    s = f.readline()
    i = int(s.strip())
except OSError as err:
    print("OS error: {0}".format(err))
except ValueError:
    print("Could not convert data to an integer.")
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

Could not convert data to an integer.


- The try ... except statement has an optional else clause, which, when present, must follow all except clauses. It is useful for code that must be executed if the try clause does not raise an exception. For example:

In [8]:
try:
    f = open('myfile.txt', 'r')
except IOError:
    print('cannot open', f.name)
else:
    print(arg, 'has', len(f.readlines()), 'lines')
    f.close()

NameError: name 'arg' is not defined

- When an exception occurs, it may have an associated value, also known as the exception’s argument.

In [9]:
try:
    raise Exception('spam', 'eggs')
except Exception as inst:
    print(type(inst))    # the exception instance
    print(inst.args)     # arguments stored in .args
    print(inst)          # __str__ allows args to be printed directly,
                         # but may be overridden in exception subclasses
    x, y = inst.args     # unpack args
    print('x =', x)
    print('y =', y)

<class 'Exception'>
('spam', 'eggs')
('spam', 'eggs')
x = spam
y = eggs


- Exception handlers don’t just handle exceptions if they occur immediately in the try clause, but also if they occur inside functions that are called (even indirectly) in the try clause. 

In [10]:
def this_fails():
    x = 1/0

try:
    this_fails()
except ZeroDivisionError as err:
    print('Handling run-time error:', err)

Handling run-time error: division by zero


## Raising Exceptions

The `raise` statement allows the programmer to force a specified exception to occur.

In [11]:
try:
    raise NameError('HiThere')
except NameError:
    print("An exception flew by! But I don't intend to handle it!")
    raise

An exception flew by! But I don't intend to handle it!


NameError: HiThere

## User-defined Exceptions

Programs may name their own exceptions by creating a new exception class. Exceptions should typically **be derived from the Exception class**, either directly or indirectly.

In [12]:
class MyError(Exception):
    def __init__(self, value):
        self.value = value
    def __str__(self):
        return repr(self.value)

try:
    raise MyError(2*2)
except MyError as e:
    print("My exception occured, value:", e.value)

My exception occured, value: 4


### Common Practice

When creating a module that can raise several distinct errors, a common practice is to **create a base class** for exceptions defined by that module, and **subclass that to create specific exception classes** for different error conditions.

Most exceptions are defined **with names that end in “Error,”** similar to the naming of the standard exceptions.

In [13]:
class Error(Exception):
    """Base class for exceptions in this module."""
    pass

class InputError(Error):
    """Exception raised for errors in the input.

    Attributes:
        expression -- input expression in which the error occurred
        message -- explanation of the error
    """

    def __init__(self, expression, message):
        self.expression = expression
        self.message = message

class TransitionError(Error):
    """Raised when an operation attempts a state transition that's not
    allowed.

    Attributes:
        previous -- state at beginning of transition
        next -- attempted new state
        message -- explanation of why the specific transition is not allowed
    """

    def __init__(self, previous, next, message):
        self.previous = previous
        self.next = next
        self.message = message

## Defining Clean-up Actions

The try statement has another optional clause which is intended to define clean-up actions that must be executed under all circumstances. 

**A finally clause is always executed before leaving the try statement, whether an exception has occurred or not. **

When an exception has occurred in the try clause and has not been handled by an except clause (or it has occurred in an except or else clause), it is **re-raised after the finally clause has been executed**.

In [14]:
def divide(x, y):
    try:
        result = x / y
    except ZeroDivisionError:
        print("division by zero!")
    else:
        print("result is", result)
    finally:
        print("executing finally clause")

In [15]:
divide(2, 1)

result is 2.0
executing finally clause


In [16]:
divide(2, 0)

division by zero!
executing finally clause


In [17]:
divide('2', '0')

executing finally clause


TypeError: unsupported operand type(s) for /: 'str' and 'str'

The finally clause **is also executed “on the way out”** when any other clause of the try statement is left via a **break, continue or return statement**. 

In [18]:
def try_with_no_return(num):
    try:
        num += 1
        print('num in try is ', num)
    except TypeError:
        num = int(num)
        num += 1
        print('num in except is ', num)
    else:
        num += 1
        print('num in else is ', num)
    finally:
        num += 1
        print('num in finally is ', num )
        return num

In [19]:
try_with_no_return(0)

num in try is  1
num in else is  2
num in finally is  3


3

In [20]:
try_with_no_return('0')

num in except is  1
num in finally is  2


2

In [21]:
def try_with_return(num):
    try:
        num += 1
        print('num in try is ', num)
        return 'try'
    except TypeError:
        num = int(num)
        num += 1
        print('num in except is ', num)
        return 'except'
    else:
        num += 1
        print('num in else is ', num)
    finally:
        num += 1
        print('num in finally is ', num )
        return 'finally'

In [22]:
try_with_return(0)

num in try is  1
num in finally is  2


'finally'

In [23]:
try_with_return('0')

num in except is  1
num in finally is  2


'finally'

In [24]:
def finally_with_no_return(num):
    try:
        num += 1
        print('num in try is ', num)
        return 'exit in try'
    except TypeError:
        num = int(num)
        num += 1
        print('num in except is ', num)
        return 'exit int except'
    else:
        num += 1
        print('num in else is ', num)
    finally:
        num += 1
        print('num in finally is ', num )

In [25]:
finally_with_no_return(0)

num in try is  1
num in finally is  2


'exit in try'

In [26]:
finally_with_no_return('0')

num in except is  1
num in finally is  2


'exit int except'

>In real world applications, the finally clause is useful for releasing external resources (such as files or network connections), regardless of whether the use of the resource was successful.

## Exception Hierarchy

For details, see [python-excptions-reference](https://docs.python.org/3/library/exceptions.html).

The class hierarchy for built-in exceptions is:

    BaseException
     +-- SystemExit
     +-- KeyboardInterrupt
     +-- GeneratorExit
     +-- Exception
          +-- StopIteration
          +-- StopAsyncIteration
          +-- ArithmeticError
          |    +-- FloatingPointError
          |    +-- OverflowError
          |    +-- ZeroDivisionError
          +-- AssertionError
          +-- AttributeError
          +-- BufferError
          +-- EOFError
          +-- ImportError
          +-- LookupError
          |    +-- IndexError
          |    +-- KeyError
          +-- MemoryError
          +-- NameError
          |    +-- UnboundLocalError
          +-- OSError
          |    +-- BlockingIOError
          |    +-- ChildProcessError
          |    +-- ConnectionError
          |    |    +-- BrokenPipeError
          |    |    +-- ConnectionAbortedError
          |    |    +-- ConnectionRefusedError
          |    |    +-- ConnectionResetError
          |    +-- FileExistsError
          |    +-- FileNotFoundError
          |    +-- InterruptedError
          |    +-- IsADirectoryError
          |    +-- NotADirectoryError
          |    +-- PermissionError
          |    +-- ProcessLookupError
          |    +-- TimeoutError
          +-- ReferenceError
          +-- RuntimeError
          |    +-- NotImplementedError
          |    +-- RecursionError
          +-- SyntaxError
          |    +-- IndentationError
          |         +-- TabError
          +-- SystemError
          +-- TypeError
          +-- ValueError
          |    +-- UnicodeError
          |         +-- UnicodeDecodeError
          |         +-- UnicodeEncodeError
          |         +-- UnicodeTranslateError
          +-- Warning
               +-- DeprecationWarning
               +-- PendingDeprecationWarning
               +-- RuntimeWarning
               +-- SyntaxWarning
               +-- UserWarning
               +-- FutureWarning
               +-- ImportWarning
               +-- UnicodeWarning
               +-- BytesWarning
               +-- ResourceWarning
               